In [58]:
import numpy as np
import pandas as pd
from datetime import datetime
import datetime
import time
import matplotlib
import base
import predict
#import holiday_tread as tread

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取休假表
calenders = base.ReadCalendarCSV();

In [60]:
%reload_ext autoreload

In [61]:
test_shop_end = 2001

In [62]:
def initWeekDate(start_time, week_num):
    time_ranges = []
    while(week_num > 0):
        end_time = start_time + datetime.timedelta(days=6)
        time_ranges.append([start_time, end_time])

        week_num = week_num - 1;
        start_time = start_time + datetime.timedelta(weeks=1)

    return time_ranges

In [63]:
def trainSingleHolidayMeanData(id, shop_open_dates, user_pay_counts, calenders, start_time, end_time):
    open_time = shop_open_dates.loc[id]['date'];
    if open_time > start_time:
        start_time = open_time
    df = base.countShopPayTimePeriods(user_pay_counts, id,
                                      date_range=[start_time, end_time], time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])

    df['holiday'] = calenders.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values

    means = {}
    for i in range(1, 6):
        if i != 4:
            temp_df = df[df['holiday'] == i];
            means[i] = temp_df['count'].mean()

    #print(mean_dict)
    #result_df = pd.DataFrame(mean_dict).T
    #result_df.rename(columns={0: 'mean', 1: 'std'}, inplace=True)

    return means


def trainHolidayMeansData(shop_open_dates, user_pay_counts, calenders, time_ranges):
    shop_mean_dict = {}
    for id in range(1,test_shop_end):
        print(id)
        mean_dict = {}
        for i, tr in enumerate(time_ranges):
            means = trainSingleHolidayMeanData(id, shop_open_dates, user_pay_counts, calenders, tr[0], tr[1])
            mean_dict[i] = means
        
        shop_mean_dict[id] = mean_dict;

    return shop_mean_dict;

In [64]:
def getHolidayMeanCount(id, day_type, shop_mean_dict, start_real, end_real, pred_mean=None):
    count = 0;
    for i in range(start_real, end_real):
        count = count + shop_mean_dict[id][i][day_type]
    
    if pred_mean != None:
        count = count + pred_mean[id][day_type]
            
    if pred_mean != None:
        return count / (end_real - start_real  + 1);
    else:
        return count / (end_real - start_real)

In [65]:
week_num = 2
time_ranges = initWeekDate(pd.to_datetime('2016-8-15'), week_num)
shop_mean_dict = trainHolidayMeansData(shop_open_dates, user_pay_counts, calenders, time_ranges)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [66]:
start_time = pd.to_datetime('2016-8-29')
end_time = pd.to_datetime('2016-9-4')
predict_date = pd.date_range(start=start_time, end=end_time, freq='D', normalize=True)
pridicts = {}
shop_pred_dict = {}
for id in range(1, test_shop_end):
    print(id)
    y_Pred = []
    pred_dict = {}
    for date in predict_date:
        daytype = calenders.loc[date.strftime('%Y-%m-%d')]['daytype']
        count = getHolidayMeanCount(id, daytype, shop_mean_dict, 0, 2)
        y_Pred.append(count);
        pred_dict[daytype] = count;
    
    shop_pred_dict[id] = pred_dict;
    pridicts[id] = y_Pred


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [67]:
start_time = pd.to_datetime('2016-9-5')
end_time = pd.to_datetime('2016-9-11')
predict_date = pd.date_range(start=start_time, end=end_time, freq='D', normalize=True)
for id in range(1, test_shop_end):
    print(id)
    y_Pred = []
    for date in predict_date:
        daytype = calenders.loc[date.strftime('%Y-%m-%d')]['daytype']
        #count = getHolidayMeanCount(id, daytype, shop_mean_dict, 0, 2)
        count = getHolidayMeanCount(id, daytype, shop_mean_dict, 1, 2, shop_pred_dict)
        pridicts[id].append(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [72]:
predict_df = pd.DataFrame(pridicts)
predict_df = predict_df.T
predict_df = predict.FixFinalPredict(predict_df)

In [73]:
start_time = pd.to_datetime('2016-8-29')
end_time = pd.to_datetime('2016-9-11')
evaluations = {}
for id in range(1, test_shop_end):
    y_Pred = predict_df.loc[id].values
    y_Test = base.getTestCount(user_pay_counts, id, start_time, end_time).values
    e = base.evaluation(y_Test, y_Pred)
    print("id=%d evaluation=%f"%(id, e))
    evaluations[id] = e

id=1 evaluation=0.029177
id=2 evaluation=0.062543
id=3 evaluation=0.089506
id=4 evaluation=0.062742
id=5 evaluation=nan
id=6 evaluation=0.086353
id=7 evaluation=0.114545
id=8 evaluation=0.096656
id=9 evaluation=0.047068
id=10 evaluation=0.325534
id=11 evaluation=0.057566
id=12 evaluation=0.062874
id=13 evaluation=0.767336
id=14 evaluation=0.142269
id=15 evaluation=0.099035
id=16 evaluation=0.099228
id=17 evaluation=0.068715
id=18 evaluation=0.153982
id=19 evaluation=0.037930
id=20 evaluation=0.085928
id=21 evaluation=0.189676
id=22 evaluation=0.130651
id=23 evaluation=0.062624
id=24 evaluation=0.296497
id=25 evaluation=0.114383
id=26 evaluation=0.160615
id=27 evaluation=0.050286
id=28 evaluation=1.000000
id=29 evaluation=0.429150
id=30 evaluation=0.089595
id=31 evaluation=0.036524
id=32 evaluation=0.204341
id=33 evaluation=0.286159
id=34 evaluation=0.031599
id=35 evaluation=0.071382
id=36 evaluation=0.173444
id=37 evaluation=0.108872
id=38 evaluation=0.073035
id=39 evaluation=0.186424


In [74]:
evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
evaluation_df.columns = ['evaluation']
evaluation_df.index.name = 'id'

In [75]:
evaluation_df['evaluation'].mean()


0.14521783551935044

In [78]:
evaluation_df.to_csv("./data/dataset/dataset/validation/mean_evaluation.txt", sep='\t', index=True, encoding='UTF-8')
predict_df.to_csv("./data/dataset/dataset/validation/mean_predict.txt", sep='\t', index=True, encoding='UTF-8')

In [ ]:
#3|3=1|1+1=1 
#0.15971531029729363
#3|3=1|2+1=1
#0.15986176343981298
#2|2=1|2=1
#0.1460136192016504
#2|2=1|1+1=1
#0.14521783551935044